<a href="https://colab.research.google.com/github/fede-m/Illuscoder/blob/main/IllusCoder_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Things to install

# 1) Install ngrok to start Flask application


# 2) Summarize text
!pip install sentencepiece


# 3) Named Entity Recognition and Corefernce
!python -m pip install spacy==2.1.0
!python -m spacy download en_core_web_md


# 4) Generate images
!pip install diffusers==0.8.0 transformers ftfy
!pip install accelerate

# 5) Modify images
!pip install Pillow

In [ ]:
# Imports
import json

# Text summarizer
from transformers import pipeline
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
import random

# Named Entity Recognition
import spacy

#Coreference
!pip uninstall neuralcorefY

!pip install neuralcoref --no-binary neuralcoref
import neuralcoref

# Stable Diffusion Models
from diffusers import DiffusionPipeline
from diffusers import DPMSolverMultistepScheduler
from transformers import pipeline
import torch

#check for gpu
use_cuda = torch.cuda.is_available()
torch.device("cuda" if use_cuda else "cpu")


#Image modification
from PIL import Image, ImageFont, ImageDraw # In case we need to modify the image
import textwrap



In [ ]:
# Load the models

# Text summarizer


summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
#tokenizer = PegasusTokenizer.from_pretrained("facebook/bart-large-cnn")
#model = PegasusForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")

# Named Entity Recognition
nlp = spacy.load('en_core_web_md')
neuralcoref.add_to_pipe(nlp)

# Image Generation
pipe = DiffusionPipeline.from_pretrained("naclbit/trinart_characters_19.2m_stable_diffusion_v1")
#pipeline = DiffusionPipeline.from_pretrained("eimiss/EimisAnimeDiffusion_1.0v")

# Scheduler for Diffusion Model ---> fastest by now is DPMSolverMultistepScheduler (Stable_Diffusions_2)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
#pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

# Generator --> Use the manual_seed attribute to select and fix a seed. This helps ensuring coherence in the style of the outputs
generator = torch.Generator().manual_seed(0)



In [ ]:
# Read the User input and save it in a dictionary
def unpack_prompt(json_file):
  story_prompt = json.load(json_file)
  return story_prompt


In [ ]:
def set_mood(genre):
  palettes = {"fantasy":["light blue, grey and white", "indigo, orange, beige", "brown, green, turquoise"], "adventure": ["brown, maroon, olive green", "gold, teal, red", "green, lavender, purple"], "romantic": ["pink, brown, gold", "red, black, gold", "silver, blue, purple"], "horror": ["black, red, white", "orange, grey, yellow", "brown, silver, red"]}
  genre += " in " + random.choice(palettes[genre])
  return genre


In [ ]:
# Summarize the text of the chapter to get only the most important parts of the story

text ="To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false position. He never spoke of the softer passions, save with a gibe and a sneer. They were admirable things for the observer—excellent for drawing the veil from men’s motives and actions. But for the trained reasoner to admit such intrusions into his own delicate and finely adjusted temperament was to introduce a distracting factor which might throw a doubt upon all his mental results. Grit in a sensitive instrument, or a crack in one of his own high-power lenses, would not be more disturbing than a strong emotion in a nature such as his. And yet there was but one woman to him, and that woman was the late Irene Adler, of dubious and questionable memory."

def summarize_text(chapter, summarizer, nlp):

  '''
  param: chapter --> string with the current chapter text
  param: summarizer --> summarization model (BART)

  output: list of max 3 summarized sentences

  '''
  # Generate summary of the chapter
  summary = summarizer(chapter, max_length=130, min_length=30, do_sample=False)
  sents = summary[0]['summary_text']


  # Pick only 3 sentences (to generate only 3 images for every chapter)
  sentences = sent_tokenize(sents)
  if len(sentences) > 3:
    sentences = random.sample(sentences, 3)

  return sentences

nlp = spacy.load('en_core_web_md')
sum_text = summarize_text(text,summarizer, nlp)
print(sum_text)

In [ ]:
# def Named Entity Recognition + References finden
# Schon ersätzen in den Sätzen

def named_entity_recognition(sentence, nlp):

  doc = nlp(sentence)
  per_entities = []
  loc_entities = []
  actions = ["","", "", ""]      #(verb, direct object, adverb, attribute)
  persons = []
  print(doc._.has_coref)
  print(doc._.coref_clusters)



  # extract characters and locations
  for ent in doc.ents:
    if ent.label_ == "PERSON":
      persons.append(ent)
    if ent.label_ == ("LOC" or "GPE" or "FAC"):
      loc_entities.append(ent)

  # extract actions
  for token in doc:

    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
    if token.dep_ == "nsubj":
      if len(persons)== 0:
        per_entities.append(token)
      else:                         #Check if Subject is a detected Person, include otherwise
          is_person = False
          for p in persons:
            if token in p:
              is_person = True
              per_entities.append(p)
          if is_person == False:
            per_entities.append(token)


    # Added negation
    if token.dep_ == "neg":
      actions[0] = token
    if token.dep_ == "ROOT":
      actions[1] = token
    if token.dep_ == "dobj":
      actions[3] = token
    if token.dep_ == "acomp" and actions[1].lemma_=="be":
      actions[2] = token
    if token.dep_ == "attr" and actions[1].lemma_=="be":
      actions[3] = token



  return per_entities, loc_entities, actions



sum_text = ["Sam and Tom go to the beach. Sam sees a shark. Sam tells Tom."]

extracted_infos = [named_entity_recognition(sent, nlp) for sent in sum_text]
print(extracted_infos)

In [ ]:
def prepare_prompts(sentences, nlp, characters, time, genre):
  prompts = []

  for sent in sentences:
    curr_prompt = ""
    char_entities, loc_entities, actions = named_entity_recognition(sent, nlp)
    for character in char_entities:
      # Need to check if the character name recognized through NER is included in the key name
      is_character = False
      for char in characters.keys():
        if str(character) in char:
          curr_prompt += str(character)+ ", " + characters[char] +", "
          is_character = True
      if not is_character:
        curr_prompt += str(character) + ", "

    for action in actions:
      curr_prompt += str(action) + " "

    for place in loc_entities:
          curr_prompt += ", " + str(place)
    curr_prompt += ", " +time
    curr_prompt += ", " + genre

    prompts.append(curr_prompt)
  return prompts
cs = {"Tom": "man, brown hair , blue eyes", "Sam": "male, black hair, brown eyes"}
prepare_prompts(sum_text, nlp, cs, "modern", set_mood("fantasy"))

In [ ]:
# Negative prompt
n_prompt = "lowres, NSFW, bad anatomy, bad hands, text, error, missing fingers, cropped, jpeg artifacts, worst quality, low quality, signature, watermark, blurry, deformed, extra ears, deformed, disfigured, mutation, censored"

In [ ]:
def generate_images(prompts):
  images = []
  for p in prompts:
    img =pipe(prompt=p, negative_prompt=n_prompt,generator=generator, num_inference_steps = 7).images[0]
    if not img.getbbox():
      print("NSFW content")
    else:
      images.append(img)
  return images

ims = generate_images(ps)
ims[0].show()

In [ ]:
ims[0]

In [ ]:
# AWS Student Developer Account free credits
# Look at "Merry the Scientist"
# Look at the seed of the Generator

In [ ]:
def add_text(img, sent):
  """Adds a sentence in on a white space at the bottom of the picture."""
  draw = ImageDraw.Draw(img)
  sent = textwrap.fill(sent, img.size[0]//6)      # Cut Sentence to the length of the picture
  text_h  = draw.textsize(sent)[1]       #get height of text
  new_size = (img.size[0], img.size[1]+text_h)
  new_image = Image.new('RGB', new_size, (255, 255, 255)) #slightly larger white background
  new_image.paste(img)

  #add text into white area
  h = new_image.size[1]                 #get height of image
  ImageDraw.Draw(new_image).text(( 0, h-text_h), sent, (0,0,0))

  #return result
  return new_image

def make_gif(images):
  """takes list of images and saves them as a gif"""
  frame_one = images[0]
  w, h = images[0].size
  ims = [i.resize((w, h)) for i in images]
  frame_one.save("my_gif.gif", format="GIF", append_images=ims, save_all=True, duration=5000, loop=0)

text_ims = []
for i in range(len(ims)):
  text_ims.append(add_text(ims[i], sum_text[i]))
make_gif(text_ims)




In [ ]:
def handler(json_file):

  # 1) create the story_prompt object by unpacking the JSON file
  story_prompt = unpack_prompt(json_file)

  # 2) summarize chapter text
  summarized_text = summarize_text(story_prompt["chapter"], summarizer, nlp)

  # 3) prepare prompt --> extract the characters description, the background etc.
  prompts = prepare_prompts(summarized_text, nlp, story_prompt["characters"], story_prompt["time"], story_prompt["genre"])



# Extract characters and places named in the chapter (for the background)
  char_entities, loc_entities = named_entity_recognition(summary, nlp)


In [ ]:
from PIL import Image

char_1 = Image.open("img1.png")
resized_char = char_1.resize((200,200))